## Collecting protocol fees across Balancer core pools on all networks
- Spreadsheet as reference: https://docs.google.com/spreadsheets/d/1xwUPpbYq7woVOU9vQ8EB8MY75I-1mauTLyDVwvKUDKo/edit#gid=0
- Collab: https://colab.research.google.com/drive/1vKCvcV5mkL1zwW3565kLSGkBEbt8NsoB?usp=sharing


In [2]:
import datetime
import os

from dotenv import load_dotenv
from web3 import Web3
from web3.middleware import geth_poa_middleware

from notebooks import get_twap_bpt_price
from notebooks.constants import ARB_CHAIN_BLOCK_TIME
from notebooks.constants import ARB_CORE_POOLS
from notebooks.constants import MAINNET_CHAIN_BLOCK_TIME
from notebooks.constants import MAINNET_CORE_POOLS
from notebooks.constants import POLYGON_CHAIN_BLOCK_TIME
from notebooks.constants import POLYGON_CORE_POOLS

load_dotenv()
arb_web3 = Web3(Web3.HTTPProvider(os.environ["ARBNODEURL"]))
eth_web3 = Web3(Web3.HTTPProvider(os.environ["ETHNODEURL"]))
poly_web3 = Web3(Web3.HTTPProvider("https://polygon-rpc.com"))
poly_web3.middleware_onion.inject(geth_poa_middleware, layer=0)

# ARBITRUM
# TODO: 
# arb_block_now = arb_web3.eth.block_number - 1000
arb_block = 122428959  # 18 August 2023
arb_timestamp_now = arb_web3.eth.get_block(arb_block).timestamp
# Given Arb block time, we want to look back 2 weeks:
arb_block_2_weeks_ago = arb_block - (2 * 7 * 24 * 60 * 60 / ARB_CHAIN_BLOCK_TIME)
arb_timestamp_2_weeks_ago = arb_web3.eth.get_block(int(arb_block_2_weeks_ago)).timestamp
# Convert to datetime:
arb_datetime_now = datetime.datetime.fromtimestamp(arb_timestamp_now)
arb_datetime_2_weeks_ago = datetime.datetime.fromtimestamp(arb_timestamp_2_weeks_ago)
# MAINNET
# TODO: 
# mainnet_block_now = eth_web3.eth.block_number - 1000
mainnet_block = 17938293
mainnet_timestamp_now = eth_web3.eth.get_block(mainnet_block).timestamp
# Given mainnet block time, we want to look back 2 weeks:
mainnet_block_2_weeks_ago = mainnet_block - (2 * 7 * 24 * 60 * 60 / MAINNET_CHAIN_BLOCK_TIME)
mainnet_timestamp_2_weeks_ago = eth_web3.eth.get_block(int(mainnet_block_2_weeks_ago)).timestamp
# Convert to datetime:
mainnet_datetime_now = datetime.datetime.fromtimestamp(mainnet_timestamp_now)
mainnet_datetime_2_weeks_ago = datetime.datetime.fromtimestamp(mainnet_timestamp_2_weeks_ago)

# POLYGON
# poly_block_now = poly_web3.eth.block_number - 1000
poly_block = 46439663
poly_timestamp_now = poly_web3.eth.get_block(poly_block).timestamp
# Given polygon block time, we want to look back 2 weeks:
poly_block_2_weeks_ago = poly_block - (2 * 7 * 24 * 60 * 60 / POLYGON_CHAIN_BLOCK_TIME)
poly_timestamp_2_weeks_ago = poly_web3.eth.get_block(int(poly_block_2_weeks_ago)).timestamp
# Convert to datetime:
poly_datetime_now = datetime.datetime.fromtimestamp(poly_timestamp_now)
poly_datetime_2_weeks_ago = datetime.datetime.fromtimestamp(poly_timestamp_2_weeks_ago)

print("ARBITRUM")
arb_bpt_twap_prices = {}
for arb_pool in ARB_CORE_POOLS:
    arb_bpt_twap_prices[arb_pool] = get_twap_bpt_price(arb_pool, 'arbitrum', arb_web3, start_date=arb_datetime_now.date(),
                                                       block_number=arb_block)
    print(arb_bpt_twap_prices[arb_pool])

print("MAINNET")
mainnet_bpt_twap_prices = {}
for mainnet_pool in MAINNET_CORE_POOLS:
    mainnet_bpt_twap_prices[mainnet_pool] = get_twap_bpt_price(mainnet_pool, 'mainnet', eth_web3,
                                                               start_date=mainnet_datetime_now.date(),
                                                               block_number=mainnet_block)
    print(mainnet_bpt_twap_prices[mainnet_pool])

print("POLYGON")
polygon_bpt_twap_prices = {}
for polygon_pool in POLYGON_CORE_POOLS:
    polygon_bpt_twap_prices[polygon_pool] = get_twap_bpt_price(polygon_pool, 'polygon', poly_web3,
                                                               start_date=poly_datetime_now.date(), block_number=poly_block)
    print(polygon_bpt_twap_prices[polygon_pool])


ARBITRUM
1840.088502140251486439064707
1849.112740619212568255530924
1840.246733687705931575477001
1.000468223339994222704040111
1.385883244842785215745603406
1839.306156742747865786560441
1.001325804118963713902572610
MAINNET
1837.999723470568645632911054
1885.826672117525479253395465
1826.533874641722987003287631
1837.840741394329575601953494
1838.533532042886070650433172
28.62142189350756253118170066
1854.658730032151914350447206
0.9829942342377910579539661548
1845.385858087329759807846652
61.57247132094396709479301097
65.83695633899880303677808390
232.3431847552983452300305698
0.8002158631486512624870551206
1848.757602533177018203754737
11.59042947158728450853396906
POLYGON
0.9995170983294423379875944963
0.9983289292298759328377664270
0.6682961564208527327482874348
0.6679009027278375185140301962
1837.899610310451431248558119


In [3]:
# Query:
POOLS_SNAPSHOTS_QUERY = """
{{
  poolSnapshots(
    first: {first}
    skip: {skip}
    orderBy: timestamp
    orderDirection: desc
    block: {{ number: {block} }}
    where: {{ protocolFee_not: null }}
  ) {{
    pool {{
      address
      id
      symbol
      totalProtocolFeePaidInBPT
      tokens {{
        symbol
        address
        paidProtocolFees
      }}
    }}
    timestamp
    protocolFee
    swapFees
    swapVolume
    liquidity
  }}
}}
"""

## Fetching data from the Balancer subgraphs

In [4]:
from notebooks.constants import POLYGON_BALANCER_GRAPH_URL
from notebooks.constants import ARB_BALANCER_GRAPH_URL
from notebooks.constants import MAINNET_BALANCER_GRAPH_URL
from typing import Dict
from typing import List
from typing import Optional

from gql import Client
from gql import gql
from gql.transport.requests import RequestsHTTPTransport
import pandas as pd


# Fetch all the data from the balancer subgraph
def make_gql_client(url: str) -> Optional[Client]:
    transport = RequestsHTTPTransport(url=url, retries=3)
    return Client(
        transport=transport, fetch_schema_from_transport=True, execute_timeout=60
    )


def get_balancer_pool_snapshots(block: int, graph_url: str) -> Optional[List[Dict]]:
    client = make_gql_client(graph_url)
    all_pools = []
    limit = 1000
    offset = 0
    while True:
        result = client.execute(
            gql(POOLS_SNAPSHOTS_QUERY.format(first=limit, skip=offset, block=block)))
        all_pools.extend(result['poolSnapshots'])
        offset += limit
        if offset >= 5000:
            break
        if len(result['poolSnapshots']) < limit - 1:
            break
    return all_pools


arbi_pool_snapshots_now = get_balancer_pool_snapshots(arb_block, ARB_BALANCER_GRAPH_URL)
arbi_pool_snapshots_2_weeks_ago = get_balancer_pool_snapshots(int(arb_block_2_weeks_ago), ARB_BALANCER_GRAPH_URL)

mainnet_pool_snapshots_now = get_balancer_pool_snapshots(mainnet_block, MAINNET_BALANCER_GRAPH_URL)
mainnet_pool_snapshots_2_weeks_ago = get_balancer_pool_snapshots(int(mainnet_block_2_weeks_ago),
                                                                 MAINNET_BALANCER_GRAPH_URL)

polygon_pool_snapshots_now = get_balancer_pool_snapshots(poly_block, POLYGON_BALANCER_GRAPH_URL)
polygon_pool_snapshots_2_weeks_ago = get_balancer_pool_snapshots(int(poly_block_2_weeks_ago),
                                                                 POLYGON_BALANCER_GRAPH_URL)

## Extract fee data from CORE pools:


In [5]:
from collections import defaultdict
from decimal import Decimal
from notebooks import fetch_token_price_balgql


def collect_fee_info(pools: list[str], chain: str, pools_now: list[dict], pools_shifted: list[dict],
                     start_date: datetime.date) -> tuple[dict, dict]:
    # Iterate through snapshots now and 2 weeks ago and extract fee data, by subtracting today's fee data from 2 weeks ago
    # and then summing across all pools
    fees = {}
    token_fees = defaultdict(list)
    for pool in pools:
        current_fees_snapshots = [x for x in pools_now if x['pool']['id'] == pool]
        current_fees_snapshots.sort(key=lambda x: x['timestamp'], reverse=True)
        fees_2_weeks_ago = [x for x in pools_shifted if x['pool']['id'] == pool]
        fees_2_weeks_ago.sort(key=lambda x: x['timestamp'], reverse=True)
        # Take first element of list, which is the most recent snapshot
        if not current_fees_snapshots or not fees_2_weeks_ago:
            continue
        pool_snapshot_now = current_fees_snapshots[0]
        pool_snapshot_2_weeks_ago = fees_2_weeks_ago[0]
        # Calculate fees
        pool_fee = float(pool_snapshot_now['protocolFee']) - float(pool_snapshot_2_weeks_ago['protocolFee'])
        pool_swap_fee = float(pool_snapshot_now['swapFees']) - float(pool_snapshot_2_weeks_ago['swapFees'])
        # Now we need to collect token fee info. Let's start with BPT tokens, which is Balancer pool token. Notice,
        # That totalProtocolFeePaidInBPT can be null, so we need to check for that
        bpt_token_fee = 0
        bpt_price_usd = arb_bpt_twap_prices[pool] if chain == 'arbitrum' else mainnet_bpt_twap_prices[
            pool] if chain == 'mainnet' else polygon_bpt_twap_prices[pool]
        if bpt_price_usd is None:
            bpt_price_usd = 0
        if pool_snapshot_now['pool']['totalProtocolFeePaidInBPT'] is not None and pool_snapshot_2_weeks_ago['pool'][
            'totalProtocolFeePaidInBPT'] is not None:
            bpt_token_fee = float(pool_snapshot_now['pool']['totalProtocolFeePaidInBPT']) - float(
                pool_snapshot_2_weeks_ago['pool']['totalProtocolFeePaidInBPT'])
            token_fees[pool_snapshot_now['pool']['symbol']].append({
                'token': pool_snapshot_now['pool']['symbol'],
                'token_fee': bpt_token_fee,
                'token_price': bpt_price_usd,
                'token_fee_in_usd': Decimal(bpt_token_fee) * bpt_price_usd,
                'token_addr': pool_snapshot_now['pool']['address'],
                'time_from': arb_datetime_2_weeks_ago,
                'time_to': arb_datetime_now,
                'chain': chain,
            })
        # Now collect fee info about fees paid in pool tokens. Pool tokens fee info is in pool.tokens dictionary. This will be separate dictionary
        else:
            bpt_price_usd = 0
            for token_data in pool_snapshot_now['pool']['tokens']:
                token_data_2_weeks_ago = \
                    [t for t in pool_snapshot_2_weeks_ago['pool']['tokens'] if t['address'] == token_data['address']][0]
                token_fee = float(token_data.get('paidProtocolFees', None)) - float(
                    token_data_2_weeks_ago.get('paidProtocolFees', None))
                # Get twap token price from CoinGecko
                token_price = fetch_token_price_balgql(token_data['address'], chain, start_date)
                token_fees[pool_snapshot_now['pool']['symbol']].append({
                    'token': token_data['symbol'],
                    'token_fee': token_fee,
                    'token_price': token_price,
                    'token_fee_in_usd': Decimal(token_fee) * token_price if token_price is not None else 0,
                    'token_addr': token_data['address'],
                    'time_from': arb_datetime_2_weeks_ago,
                    'time_to': arb_datetime_now,
                    'chain': chain,
                })
        # Calculate non-BPT fees in USD
        fees[pool_snapshot_now['pool']['symbol']] = {
            'pool_fee': round(pool_fee, 2),
            'swap_fee': round(pool_swap_fee, 2),
            'bpt_token_fee': round(bpt_token_fee, 2),
            # Get fee in USD by multiplying bpt_token_fee by price of BPT token taken from twap_bpt_price
            'bpt_token_fee_in_usd': round(Decimal(bpt_token_fee) * bpt_price_usd, 2),
            'token_fees_in_usd': round(sum([x['token_fee_in_usd'] for x in
                                      token_fees[pool_snapshot_now['pool']['symbol']]]) if bpt_price_usd == 0 else 0, 2),
            'time_from': arb_datetime_2_weeks_ago,
            'time_to': arb_datetime_now,
            'chain': chain,
            'token_fees': token_fees[pool_snapshot_now['pool']['symbol']],
        }
    return fees


arb_fees = collect_fee_info(ARB_CORE_POOLS, 'arbitrum', arbi_pool_snapshots_now,
                            arbi_pool_snapshots_2_weeks_ago, arb_datetime_now.date())
mainnet_fees = collect_fee_info(MAINNET_CORE_POOLS, 'mainnet', mainnet_pool_snapshots_now,
                                mainnet_pool_snapshots_2_weeks_ago, mainnet_datetime_now.date())
polygon_fees = collect_fee_info(POLYGON_CORE_POOLS, 'polygon', polygon_pool_snapshots_now,
                                polygon_pool_snapshots_2_weeks_ago, poly_datetime_now.date())
# Convert to dataframe, sort by chain and pool fee
joint_fees = {**arb_fees, **mainnet_fees, **polygon_fees}
joint_fees_df = pd.DataFrame.from_dict(joint_fees, orient='index')

In [6]:
# Remove `token_fees` field from dataframe, as it's too big
joint_fees_df_copy = joint_fees_df.drop(columns=['token_fees'])
# Display all rows in dataframe
pd.set_option('display.max_rows', 1000)
joint_fees_df_copy.sort_values(by=['chain', 'pool_fee'], ascending=False)

,pool_fee,swap_fee,bpt_token_fee,bpt_token_fee_in_usd,token_fees_in_usd,time_from,time_to,chain
MaticX-bb-a-WMATIC-BPT,2653.65,295.61,3907.19,2609.62,0,2023-08-04 06:57:36,2023-08-18 03:24:52,polygon
stMATIC-bb-a-WMATIC-BPT,2096.43,88.94,0.00,0.00,0.00,2023-08-04 06:57:36,2023-08-18 03:24:52,polygon
wstETH-bb-a-WETH-BPT,0.00,0.02,0.00,0.00,0.00,2023-08-04 06:57:36,2023-08-18 03:24:52,polygon
bb-am-usd,0.00,0.00,0.00,0.00,0.00,2023-08-04 06:57:36,2023-08-18 03:24:52,polygon
wstETH-rETH-sfrxETH-BPT,29879.20,4302.87,15.95,29321.55,0,2023-08-04 06:57:36,2023-08-18 03:24:52,mainnet
B-rETH-STABLE,27610.34,16875.81,0.00,0.00,27161.41,2023-08-04 06:57:36,2023-08-18 03:24:52,mainnet
50STG-50bbaUSD,16308.63,27735.48,20756.67,16609.82,0,2023-08-04 06:57:36,2023-08-18 03:24:52,mainnet
50rETH-50BADGER,12131.48,12205.55,174.04,11458.30,0,2023-08-04 06:57:36,2023-08-18 03:24:52,mainnet
swETH-bb-a-WETH-BPT,7081.73,35.44,3.80,6949.77,0,2023-08-04 06:57:36,2023-08-18 03:24:52,mainnet
ankrETH/wstETH,6871.85,869.23,3.70,6835.76,0,2023-08-04 06:57:36,2023-08-18 03:24:52,mainnet


## Now let's calculate bribes paid to the pools

In [30]:
from notebooks import calculate_aura_vebal_share

aura_vebal_share = calculate_aura_vebal_share(eth_web3, mainnet_block)

# Bribes are split per chain and per pool, with each pool getting a share of the bribe proportional to its share of fees
# paid by all pools on that chain. For example, if pool A paid 10% of all fees on Arbitrum, it will get 10% of the bribes. That 10% will be distributed between aura and vebal, proportional to their share of the bribe.
FEE = Decimal(0.5)  # 50% goes to fees
# Let's calculate share of fees paid by each pool on each chain
def calc_and_split_bribes(fees: Dict, chain: str) -> Dict[str, Dict]:
    pool_bribs = {}
    for pool, data in fees.items():
        pool_fees = data['bpt_token_fee_in_usd'] + data['token_fees_in_usd']
        # Split fees between aura and bal fees
        pool_bribs[pool] = {
            "chain": chain,
            "aura_bribes": pool_fees * FEE * aura_vebal_share,
            "bal_bribes": pool_fees * FEE * (1 - aura_vebal_share),
            "pool_total": pool_fees,
            "fees_taken": pool_fees * FEE,
        }
    return pool_bribs
mainnet_bribes = calc_and_split_bribes(mainnet_fees, 'mainnet')
arb_bribes = calc_and_split_bribes(arb_fees, 'arbitrum')
polygon_bribes = calc_and_split_bribes(polygon_fees, 'polygon')
# Convert to dataframe
joint_bribes_data = {**arb_bribes, **mainnet_bribes, **polygon_bribes}
joint_bribes_df = pd.DataFrame.from_dict(joint_bribes_data, orient='index')
# Sort by chain
joint_bribes_df.sort_values(by=['chain'], ascending=False)
# Dump into csv
joint_bribes_df.to_csv('bribes.csv')
joint_bribes_df

,chain,aura_bribes,bal_bribes,pool_total,fees_taken
rETH-bb-a-WETH-BPT,arbitrum,226.5549840816203747886574223,385.9850159183796252113425777,1225.08,612.540
B-wstETH-WETH-Stable,arbitrum,911.5792299552121603860393049,1553.070770044787839613960695,4929.30,2464.650
wstETH/rETH/cbETH,arbitrum,0E-31,0E-31,0.00,0.000
bb-a-USD,arbitrum,606.0107263493867366763303838,1032.469273650613263323669616,3276.96,1638.480
RDNT-WETH,arbitrum,3165.935211442669637591549896,5393.849788557330362408450104,17119.57,8559.785
wstETH-bb-a-WETH-BPT,polygon,0E-31,0E-31,0.00,0.000
bbaUSDC/bbaUSDCe,arbitrum,58.69887476519665912566342803,100.0061252348033408743365720,317.41,158.705
B-rETH-STABLE,mainnet,5022.980385104943729379622226,8557.724614895056270620377774,27161.41,13580.705
swETH-bb-a-WETH-BPT,mainnet,1285.226296830348085100538490,2189.658703169651914899461510,6949.77,3474.885
cbETH-bb-a-WETH-BPT,mainnet,0E-31,0E-31,0.00,0.000
